# Advanced Data Analytics for Management Support - Assignment Submission

## Table of Contents

        1. Introduction
        2. Model Selection
            2.1 Data preparation
            2.2 RNN Architecture (unstructured data)
            2.2.1 LSTM
            2.2.2 Bidirectional LSTM
            2.2.3 CNN-LSTM
        3. Full Model Architecture


## 1. Introduction

In this auxillary code file we have trained NLP models using self-trained word embeddings.The results from this notebook are referred in the main notebook "w2v pretrained modeling airbnb.ipnyb". 

The task of this assignment is to predict the price of propertied hosted on an online marketplace for accommodation rental called as Airbnb, based on different webscraped features like the location, number of rooms, property type, texts with the listing description, and rental price per night.The main focus hereby lies on building an NN-based model that predicts the price of listings in the unseen dataset based on both text and tabular data.


In [1]:
!pip install textstat

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 1.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 7.9 MB/s eta 0:00:00a 0:00:01


In [2]:
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
import re
import datetime
import time
import textstat
from bs4 import BeautifulSoup
from collections import Counter
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from gensim.models import KeyedVectors
from gensim.models.keyedvectors import Word2VecKeyedVectors
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import Input, concatenate, Dense, Embedding, LSTM, GRU, Bidirectional, BatchNormalization, Dropout
from keras.layers.embeddings import Embedding
from keras.initializers import Constant
from tensorflow.keras import activations, losses
from keras.callbacks import EarlyStopping
from tensorflow.keras import layers, models, optimizers, losses
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import numpy as np # linear algebra
import pandas as pd
from bs4 import BeautifulSoup
import nltk  

from tensorflow.keras import layers, models, optimizers, losses
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from numpy import mean
from numpy import absolute
from numpy import sqrt


import tensorflow as tf
import string
import statistics
import math
import warnings

warnings.filterwarnings('ignore')


nltk.download('omw-1.4')
# When running this notebook for the first time, you have to download some NLTK packages. To do so, simply uncomment the next lines
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
!pip install gensim==4.2.0 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 28.4 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: gensim
    Found existing installation: gensim 4.0.1
    Uninstalling gensim-4.0.1:
      Successfully uninstalled gensim-4.0.1


In [4]:
train = pd.read_pickle('../input/airbnb-last-version-numerical-data/x_train.pkl')
test = pd.read_pickle('../input/airbnb-last-version-numerical-data/x_test.pkl')

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55284 entries, 0 to 55283
Data columns (total 48 columns):
 #   Column                                 Non-Null Count  Dtype         
---  ------                                 --------------  -----         
 0   host_is_superhost                      55284 non-null  float64       
 1   reviews_per_month                      55284 non-null  float64       
 2   room_type_Private room                 55284 non-null  uint8         
 3   bed_type_Real Bed                      55284 non-null  uint8         
 4   bed_type_Pull-out Sofa                 55284 non-null  uint8         
 5   bedrooms                               55284 non-null  float64       
 6   bed_type_Couch                         55284 non-null  uint8         
 7   beds                                   55284 non-null  float64       
 8   room_type_Shared room                  55284 non-null  uint8         
 9   review_scores_location                 55284 non-null  float6

In [6]:
train.isnull().sum()/55284 * 100


host_is_superhost                         0.000000
reviews_per_month                         0.000000
room_type_Private room                    0.000000
bed_type_Real Bed                         0.000000
bed_type_Pull-out Sofa                    0.000000
bedrooms                                  0.000000
bed_type_Couch                            0.000000
beds                                      0.000000
room_type_Shared room                     0.000000
review_scores_location                    0.000000
guests_included                           0.000000
bed_type_Futon                            0.000000
review_scores_rating                      0.000000
house_rules                              42.287099
summary                                   5.343318
host_identity_verified                    0.000000
host_has_profile_pic                      0.000000
review_scores_accuracy                    0.000000
host_response_time_within a day           0.000000
space                          

## 2. Model Selection

### 2.1 Data Preparation 

This notebook covers the embedding layer experimentation using Textvectorization form keras. We start with handling the null values in the text columns by replacing them with empty strings.

Looking at the "description" feature we can see that it is a combination of summary,space and neighborhood_overview text features. In order to avoid redundant information in the pipeline, we chose to drop the description feature from the analysis. Also, since "description" feature is relatively long as compared to summary,space and neighborhood_overview features, the context might get lost in dense layers of the NLP models. 

In [7]:
train['neighborhood_overview'].fillna('',inplace=True)
train['space'].fillna('',inplace=True)
train['summary'].fillna('',inplace=True)
train['name'].fillna('',inplace=True)
train['transit'].fillna('',inplace=True)
train['house_rules'].fillna('',inplace=True)
train['host_response_rate'].fillna(0,inplace=True)

In [8]:
test['neighborhood_overview'].fillna('',inplace=True)
test['space'].fillna('',inplace=True)
test['summary'].fillna('',inplace=True)
test['name'].fillna('',inplace=True)
test['transit'].fillna('',inplace=True)
test['house_rules'].fillna('',inplace=True)
test['host_response_rate'].fillna(0,inplace=True)

In [9]:
text_cols = ['name', 'summary', 'space', 'neighborhood_overview', 'transit', 'house_rules','ameneties']

In [10]:
train_name = train['name']
train_summary = train['summary']
train_space = train['space']
train_ngbr = train['neighborhood_overview']
train_transit = train['transit']
train_hr = train['house_rules']
y_train = train['price']
print(len(train_name), len(train_summary), len(y_train))

55284 55284 55284


In [11]:
test_name = test['name']
test_summary = test['summary']
test_space = test['space']
test_ngbr = test['neighborhood_overview']
test_transit = test['transit']
test_hr = test['house_rules']
print(len(test_name), len(test_summary))

29769 29769


The number of hidden units in the NLP models is chosen to be 128. The embedding size is also fixed to 100 and the output sequence length is set to 500. 

In [12]:
vocab_size = 10000
seq_length = 500
emb_size = 100
rnn_units = 128
def our_standardization(text_data):
  lowercase = tf.strings.lower(text_data) # convert to lowercase
  remove_html = tf.strings.regex_replace(lowercase, '<br />', ' ') # remove HTML tags
  pattern_remove_punctuation = '[%s]' % re.escape(string.punctuation) # pattern to remove punctuation
  remove_punct = tf.strings.regex_replace(remove_html, pattern_remove_punctuation, '') # apply pattern
  remove_double_spaces = tf.strings.regex_replace(remove_punct, '\s+', ' ') # remove double space
  return remove_double_spaces

# Create a vectorization layer
vectorize_layer = TextVectorization(
    standardize = our_standardization,
    max_tokens = vocab_size,
    output_sequence_length = seq_length
    )

2022-09-09 21:12:24.228920: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-09 21:12:24.364358: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-09 21:12:24.365620: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-09 21:12:24.368361: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

## 2.2 RNN Architecture (unstructured data)

In [13]:
# Extract target variable and feature matrix 
X = train.drop(['price'], axis=1) 
y = train[['price']]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.2, random_state=888)

In [14]:
x_train_num_cols = X_train.select_dtypes(exclude=["category","datetime64[ns]","object"])
x_val_num_cols = X_val.select_dtypes(exclude=["category","datetime64[ns]","object"])
x_train_num_cols.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44227 entries, 25103 to 10750
Data columns (total 33 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   host_is_superhost                      44227 non-null  float64
 1   reviews_per_month                      44227 non-null  float64
 2   room_type_Private room                 44227 non-null  uint8  
 3   bed_type_Real Bed                      44227 non-null  uint8  
 4   bed_type_Pull-out Sofa                 44227 non-null  uint8  
 5   bedrooms                               44227 non-null  float64
 6   bed_type_Couch                         44227 non-null  uint8  
 7   beds                                   44227 non-null  float64
 8   room_type_Shared room                  44227 non-null  uint8  
 9   review_scores_location                 44227 non-null  float64
 10  guests_included                        44227 non-null  float64
 11

Since there are so many possibilities for an NLP model, we limit ourselves to trying 4 different model configurations on the text features. For all of them, we decide on some kind of LSTM architecture with embedding layer using Textvectorization.
The Keras Sequential API allows us to effectively train text feature models.

1. Basic GRU
2. LSTM
3. CNN-LSTM
4. Bidirectional LSTM

For all of these models, different hyperparameter configurations were tested. The number of hidden units inside the GRU units does not seem to make a big difference, so they are left at 128 for all the other models. The final activation functions are set to ReLU, since clap counts are never negative.

We refrain from using mean squared error loss, since it would make training performance difficult to judge.
Instead, we could use the mean absolute error, but this loss can have difficulties finding a minimum because of its shape, especially at the high batch size of 64.

We therefore decide for log-cosh loss, which behaves very similarly to MAE, but does not suffer as much from the aforementioned negative property.

In [15]:
x_train_name = X_train['name']
x_train_summary = X_train['summary']
x_train_space = X_train['space']
x_train_ngbr = X_train['neighborhood_overview']
x_train_transit = X_train['transit']
x_train_hr = X_train['house_rules']
x_train_amenities = X_train['amenities']
print(len(x_train_name), len(x_train_summary), len(y_val))

44227 44227 11057


In [16]:
x_val_name = X_val['name']
x_val_summary = X_val['summary']
x_val_space = X_val['space']
x_val_ngbr = X_val['neighborhood_overview']
x_val_transit = X_val['transit']
x_val_hr = X_val['house_rules']
x_val_amenities = X_val['amenities']
print(len(x_val_name), len(x_val_summary), len(y_val))

11057 11057 11057


#### 2.2.1 LSTM


In [36]:
def create_text_model(text_list):
    model = models.Sequential()
    model.add(layers.Input(shape=(1,), dtype=tf.string))
    vectorize_layer.adapt(text_list)
    model.add(vectorize_layer)
    model.add(layers.Embedding(input_dim=vocab_size, output_dim=emb_size))
    model.add(layers.LSTM(rnn_units, return_sequences=False))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(1, activation="relu"))
    return model

def create_combined_model(X):
    X = layers.Flatten()(X)
    X = layers.Dense(1, activation="relu")(X)
    print(X.shape)
    return X


def create_model():
    
    ### Initialize Input layers
    input_desc = layers.Input(shape=(1,), dtype=tf.string)
    input_ngbr = layers.Input(shape=(1,), dtype=tf.string)
    input_name = layers.Input(shape=(1,), dtype=tf.string)
    input_space = layers.Input(shape=(1,), dtype=tf.string)
      
    
    ### Create Vectorisation models from text features
    desc_model = create_text_model(x_train_summary)
    ngbr_model = create_text_model(x_train_ngbr)
    name_model = create_text_model(x_train_name)
    space_model = create_text_model(x_train_space) 
    
    ### Create Data flow
    emb_desc = desc_model(input_desc)
    emb_ngbr = ngbr_model(input_ngbr)
    emb_name = desc_model(input_name)
    emb_space = ngbr_model(input_space)
    concat_combined = layers.Concatenate()([emb_desc,emb_ngbr,emb_name,emb_space])
    print(concat_combined.shape)
    output = create_combined_model(concat_combined)
    
    ### Finalize the model
    model = tf.keras.Model(inputs = [input_desc, input_ngbr,input_name,input_space
                                     ], outputs = output)
    model.compile(optimizer=tf.optimizers.Adam(learning_rate=1e-4),loss='mse', metrics=['mae'])
    return model


model = create_model()
print(model.summary())

history = model.fit(
    [x_train_summary, x_train_ngbr,x_train_name,x_train_space],
    y_train,
    validation_split=0.2,
    epochs = 5,
    batch_size = 32,
    verbose =1)

(None, 4)
(None, 1)
Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_61 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_62 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_63 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_64 (InputLayer)           [(None, 1)]          0                                            
________________________________________________________________________

In [37]:
preds_val = model.predict([x_val_summary,x_val_ngbr,x_val_name,x_val_space])
print(f"MSE: {mean_squared_error(y_val, preds_val)}")
print(f"MAE: {mean_absolute_error(y_val, preds_val)}")

MSE: 7121.430842689313
MAE: 58.98648451965283


The model already starts overfitting at the fourth epoch. If this trend carries over to our final model, we should be prepared for using regularization techniques.

#### 2.2.2 CNN-LSTM

In [17]:
def create_text_model(text_list):
    model = models.Sequential()
    model.add(layers.Input(shape=(1,), dtype=tf.string))
    vectorize_layer.adapt(text_list)
    model.add(vectorize_layer)
    model.add(layers.Embedding(input_dim=vocab_size, output_dim=emb_size))
    model.add(layers.Conv1D(filters=16,kernel_size=5))
    model.add(layers.AveragePooling1D(pool_size=2,strides=2))
    model.add(layers.Dropout(0.2))
    model.add(layers.LSTM(rnn_units, return_sequences=True))
    model.add(layers.Dense(1, activation="relu"))
    return model

def create_combined_model(X):
    X = layers.Flatten()(X)
    X = layers.Dense(128, activation="relu")(X)
    X = layers.Dense(1, activation="relu")(X)
    print(X.shape)
    return X


def create_model():
    ### Initialize Input layers
    input_desc = layers.Input(shape=(1,), dtype=tf.string)
    input_ngbr = layers.Input(shape=(1,), dtype=tf.string)
    input_name = layers.Input(shape=(1,), dtype=tf.string)
    input_space = layers.Input(shape=(1,), dtype=tf.string)
    
    
    ### Create Vectorisation models from text features
    desc_model = create_text_model(x_train_summary)
    ngbr_model = create_text_model(x_train_ngbr)
    name_model = create_text_model(x_train_name)
    space_model = create_text_model(x_train_space) 
    
    ### Create Data flow
    emb_desc = desc_model(input_desc)
    emb_ngbr = ngbr_model(input_ngbr)
    emb_name = desc_model(input_name)
    emb_space = ngbr_model(input_space)
    concat_combined = layers.Concatenate()([emb_desc,emb_ngbr,emb_name,emb_space])
    print(concat_combined.shape)
    output = create_combined_model(concat_combined)
    
    ### Finalize the model
    model = tf.keras.Model(inputs = [input_desc, input_ngbr,input_name,input_space], outputs = output)
    model.compile(optimizer=tf.optimizers.Adam(learning_rate=1e-4),
                  loss=losses.LogCosh(),
                  metrics=['mae'])
    return model


model = create_model()
print(model.summary())
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.01, verbose=0, patience=3, restore_best_weights=True)

history = model.fit(
    [x_train_summary, x_train_ngbr,x_train_name,x_train_space],
    y_train,
    validation_split=0.2,
    epochs = 30,
    batch_size = 64,
    callbacks=[earlystopping],
    verbose =1)

2022-09-09 21:13:11.294505: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


(None, 248, 4)
(None, 1)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 1)]          0                                            
_____________________________________________________________________

2022-09-09 21:13:34.575994: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


553/553 [==============================] - 61s 88ms/step - loss: 58.9103 - mae: 59.5982 - val_loss: 49.8242 - val_mae: 50.5105
Epoch 2/30
553/553 [==============================] - 46s 84ms/step - loss: 41.1054 - mae: 41.7884 - val_loss: 39.6246 - val_mae: 40.3072
Epoch 3/30
553/553 [==============================] - 47s 85ms/step - loss: 38.1375 - mae: 38.8195 - val_loss: 38.1029 - val_mae: 38.7852
Epoch 4/30
553/553 [==============================] - 48s 86ms/step - loss: 36.6997 - mae: 37.3809 - val_loss: 37.3446 - val_mae: 38.0260
Epoch 5/30
553/553 [==============================] - 48s 87ms/step - loss: 35.6255 - mae: 36.3063 - val_loss: 37.8951 - val_mae: 38.5772
Epoch 6/30
553/553 [==============================] - 48s 87ms/step - loss: 34.6473 - mae: 35.3280 - val_loss: 36.3255 - val_mae: 37.0073
Epoch 7/30
553/553 [==============================] - 48s 86ms/step - loss: 33.7037 - mae: 34.3834 - val_loss: 36.1271 - val_mae: 36.8077
Epoch 8/30
553/553 [=========================

In [54]:
preds_val = model.predict([x_val_summary,x_val_ngbr,x_val_name,x_val_space])
print(f"MSE: {mean_squared_error(y_val, preds_val)}")
print(f"MAE: {mean_absolute_error(y_val, preds_val)}")

MSE: 3550.329466919092
MAE: 36.57255240936569


Using CNN-LSTMs instead of LSTMs and fine-tuneable hyper-parameters like learning rate and epochs does already produce better performance. Especially notable is that this model starts to overfit much later than LSTM. 

#### 2.2.3 Bidirectional LSTM

In [49]:
def create_text_model(text_list):
    model = models.Sequential()
    model.add(layers.Input(shape=(1,), dtype=tf.string))
    vectorize_layer.adapt(text_list)
    model.add(vectorize_layer)
    model.add(layers.Embedding(input_dim=vocab_size, output_dim=emb_size))
    model.add(layers.Bidirectional(layers.LSTM(rnn_units, return_sequences=True)))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(1, activation="relu"))
    return model

def create_combined_model(X):
    X = layers.Flatten()(X)
    X = layers.Dense(128, activation="relu")(X)
    X = layers.Dense(1, activation="linear")(X)
    print(X.shape)
    return X


def create_model():
    
    ### Initialize Input layers
    input_desc = layers.Input(shape=(1,), dtype=tf.string)
    input_ngbr = layers.Input(shape=(1,), dtype=tf.string)
    input_name = layers.Input(shape=(1,), dtype=tf.string)
    input_space = layers.Input(shape=(1,), dtype=tf.string)
    
    desc_model = create_text_model(x_train_summary)
    ngbr_model = create_text_model(x_train_ngbr)
    name_model = create_text_model(x_train_name)
    space_model = create_text_model(x_train_space)
    #     combined_model = create_combined_model()
    
    ### Create Data flow
    emb_desc = desc_model(input_desc)
    emb_ngbr = ngbr_model(input_ngbr)
    emb_name = desc_model(input_name)
    emb_space = ngbr_model(input_space)
    concat_combined = layers.Concatenate()([emb_desc,emb_ngbr,emb_name,emb_space])
    print(concat_combined.shape)
    output = create_combined_model(concat_combined)
    
    ### Finalize the model
    model = tf.keras.Model(inputs = [input_desc, input_ngbr,input_name,input_space], outputs = output)
    model.compile(optimizer=tf.optimizers.Adam(learning_rate=1e-5),
                  loss=losses.LogCosh(),
                  metrics=['mae'])
    return model



model = create_model()
print(model.summary())
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.01, verbose=0, patience=3, restore_best_weights=True)

history = model.fit(
    [x_train_summary, x_train_ngbr,x_train_name,x_train_space],
    y_train,
    validation_split=0.2,
    epochs = 30,
    batch_size = 64,
    callbacks=[earlystopping],
    verbose =1)

(None, 500, 4)
(None, 1)
Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_140 (InputLayer)          [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_141 (InputLayer)          [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_142 (InputLayer)          [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_143 (InputLayer)          [(None, 1)]          0                                            
__________________________________________________________________

In [52]:
preds_val = model.predict([x_val_summary,x_val_ngbr,x_val_name,x_val_space])
print(f"MSE: {mean_squared_error(y_val, preds_val)}")
print(f"MAE: {mean_absolute_error(y_val, preds_val)}")

MSE: 3421.79965495661
MAE: 34.791717146613166


# 3. Full model architecture

In [24]:
def create_ann():
    model = models.Sequential()
    model.add(layers.Dense(16, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(8, activation='relu'))
    return model

def create_text_model(text_list):
    model = models.Sequential()
    model.add(layers.Input(shape=(1,), dtype=tf.string))
    vectorize_layer.adapt(text_list)
    model.add(vectorize_layer)
    model.add(layers.Embedding(input_dim=vocab_size, output_dim=emb_size))
    model.add(layers.Conv1D(filters=16,kernel_size=5))
    model.add(layers.AveragePooling1D(pool_size=2,strides=2))
    model.add(layers.Dropout(0.2))
    model.add(layers.LSTM(rnn_units, return_sequences=True))
    model.add(layers.Dense(1, activation="relu"))
    return model

def create_combined_model(X):
    X = layers.Dropout(0.2)(X)
    X = layers.Dense(128, activation="relu")(X)
    # X = layers.Dense(64, activation="relu")(X)
    X = layers.Dense(1, activation="relu")(X)
    print(X.shape)
    return X



def create_model():
    
    ### Initialize Input layers
    input_desc = layers.Input(shape=(1,), dtype=tf.string)
    input_ngbr = layers.Input(shape=(1,), dtype=tf.string)
    input_name = layers.Input(shape=(1,), dtype=tf.string)
    input_space = layers.Input(shape=(1,), dtype=tf.string)
    input_numeric = tf.keras.Input(shape=(x_train_num_cols.shape[1],), dtype=tf.float64, name="numeric")
    
    ### Create Vectorisation models from text features
    desc_model = create_text_model(x_train_summary)
    ngbr_model = create_text_model(x_train_ngbr)
    name_model = create_text_model(x_train_name)
    space_model = create_text_model(x_train_space)
#     combined_model = create_combined_model()
    
    ### Create Data flow
    emb_desc = desc_model(input_desc)
    emb_ngbr = ngbr_model(input_ngbr)
    emb_name = desc_model(input_name)
    emb_space = ngbr_model(input_space)
    numeric_layers = create_ann()(input_numeric)
    concat_combined = layers.Concatenate()([emb_desc,emb_ngbr,emb_name,emb_space])
    print(concat_combined.shape)
    concat_combined = layers.Flatten()(concat_combined)
    concat_combined = layers.Concatenate()([concat_combined, numeric_layers])
    output = create_combined_model(concat_combined)
    
    ### Finalize the model
    model = tf.keras.Model(inputs = [input_desc, input_ngbr,input_name,input_space,input_numeric], outputs = output)
    model.compile(optimizer=tf.optimizers.Adam(learning_rate=1e-4),
                  loss=losses.LogCosh(),
                  metrics=['mae'])
    return model

model = create_model()
print(model.summary())
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.01, verbose=0, patience=3, restore_best_weights=True)

history = model.fit(
    [x_train_summary, x_train_ngbr,x_train_name,x_train_space,x_train_num_cols],
    y,
    validation_split=0.2,
    epochs = 30,
    batch_size = 64,
    callbacks=[earlystopping],
    verbose =1)

(None, 248, 4)
(None, 1)
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_14 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_15 (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_16 (InputLayer)           [(None, 1)]          0                                            
___________________________________________________________________

In [25]:
preds_val = model.predict([x_val_summary,x_val_ngbr,x_val_name,x_val_space,x_val_num_cols])
print(f"MSE: {mean_squared_error(y_val, preds_val)}")
print(f"MAE: {mean_absolute_error(y_val, preds_val)}")

MSE: 7756.347374607242
MAE: 57.25877130243978


It can be seen from all the above models that the self-training of word embeddings doesn't perfom well both in terms of MAE and also results in overfitting of train data.